In [13]:
import soundfile as sf
from pytube import YouTube
from pydub import AudioSegment
import noisereduce as nr
import librosa
import pandas as pd
from googleapiclient.discovery import build
import os
import json
import subprocess
import librosa
import os
## ../AppSettings.py import 
import sys
sys.path.append("..")
from AppSettings import AppSetting

app_settings = AppSetting()




/mnt/d/GITHUB/AbdullahAliGUN/AI-Cover-Classification/DataTransactions


In [14]:

def tr_char_to_non_tr_char(text):
    tr = "çğıöşüÇĞİÖŞÜ"
    non_tr = "cgiosuCGIOSU"
    for i in range(len(tr)):
        text = text.replace(tr[i], non_tr[i])
    ## Remove special characters
    text = ''.join(e for e in text if e.isalnum() or e == ' ')

    return text

In [15]:
import csv

def append_to_csv(link, video_title, tag='AI COVER', file_name='youtube_videos.csv'):
    # Dosyanın var olup olmadığını kontrol et ve başlıkları ekle
    try:
        with open(file_name, 'r', newline='', encoding='utf-8') as file:
            existing_videos = [row for row in csv.reader(file)]
            links_titles = [row[:2] for row in existing_videos]  # Link ve başlık sütunlarını al
    except FileNotFoundError:
        existing_videos = []
        links_titles = []

    # Video daha önce eklenmiş mi kontrol et
    if [link, video_title] not in links_titles:
        with open(file_name, 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            if not existing_videos:  # Eğer dosya boşsa, başlıkları yaz
                writer.writerow(['Link', 'Video Adı', 'Etiket'])
            writer.writerow([link, video_title, tag])
            print(f"{video_title} CSV'ye eklendi.")

In [16]:
def search_youtube(query,tag):
    youtube = build('youtube', 'v3', developerKey=app_settings.get_secret()["GOOGLE"]["API_KEY"])
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=20,
    )
    response = request.execute()
    for item in response['items']:
        video_id = item['id']['videoId']
        video_title = tr_char_to_non_tr_char(item['snippet']['title'])
        video_link = f"https://www.youtube.com/watch?v={video_id}"
        print(f"Title: {video_title}\nLink: {video_link}\n")
        
        # Eğer video_indir_ve_wav_donustur ve sesi_isle_ve_bol fonksiyonları bir WAV dosyası döndürürse,
        # bunlar burada çağrılır.
        
        append_to_csv(video_link, video_title,tag)  # CSV'ye ekleme işlemi

In [17]:
"""search_youtube("blackpink music ", "NON AI COVER")
search_youtube("barış manço müzik ", "NON AI COVER")
search_youtube("zekimüren muzik ", "NON AI COVER")
search_youtube("tarkan muzik ", "NON AI COVER")
search_youtube("sezen aksu muzik ", "NON AI COVER")
search_youtube("linkinpark music ", "NON AI COVER")
search_youtube("eminem music ", "NON AI COVER")
search_youtube("cem karaca music ", "NON AI COVER")
search_youtube("pink floyd music ", "NON AI COVER")
search_youtube("queen music ", "NON AI COVER")
search_youtube("metallica music ", "NON AI COVER")
search_youtube("nirvana music ", "NON AI COVER")"""

'search_youtube("blackpink music ", "NON AI COVER")\nsearch_youtube("barış manço müzik ", "NON AI COVER")\nsearch_youtube("zekimüren muzik ", "NON AI COVER")\nsearch_youtube("tarkan muzik ", "NON AI COVER")\nsearch_youtube("sezen aksu muzik ", "NON AI COVER")\nsearch_youtube("linkinpark music ", "NON AI COVER")\nsearch_youtube("eminem music ", "NON AI COVER")\nsearch_youtube("cem karaca music ", "NON AI COVER")\nsearch_youtube("pink floyd music ", "NON AI COVER")\nsearch_youtube("queen music ", "NON AI COVER")\nsearch_youtube("metallica music ", "NON AI COVER")\nsearch_youtube("nirvana music ", "NON AI COVER")'

In [18]:
"""search_youtube("blackpink AI COVER", "AI COVER")
search_youtube("barış manço AI COVER", "AI COVER")
search_youtube("zekimüren AI COVER", "AI COVER")
search_youtube("tarkan AI COVER", "AI COVER")
search_youtube("sezen aksu AI COVER", "AI COVER")
search_youtube("linkinpark AI COVER", "AI COVER")
search_youtube("eminem AI COVER", "AI COVER")
search_youtube("cem karaca AI COVER", "AI COVER")
search_youtube("pink floyd AI COVER", "AI COVER")
search_youtube("queen AI COVER", "AI COVER")
search_youtube("metallica AI COVER", "AI COVER")
search_youtube("nirvana AI COVER", "AI COVER")"""


'search_youtube("blackpink AI COVER", "AI COVER")\nsearch_youtube("barış manço AI COVER", "AI COVER")\nsearch_youtube("zekimüren AI COVER", "AI COVER")\nsearch_youtube("tarkan AI COVER", "AI COVER")\nsearch_youtube("sezen aksu AI COVER", "AI COVER")\nsearch_youtube("linkinpark AI COVER", "AI COVER")\nsearch_youtube("eminem AI COVER", "AI COVER")\nsearch_youtube("cem karaca AI COVER", "AI COVER")\nsearch_youtube("pink floyd AI COVER", "AI COVER")\nsearch_youtube("queen AI COVER", "AI COVER")\nsearch_youtube("metallica AI COVER", "AI COVER")\nsearch_youtube("nirvana AI COVER", "AI COVER")'

In [19]:
def convert_to_wav(input_file_path):
    """
    Verilen ses dosyasını FFmpeg kullanarak WAV formatına dönüştürür.
    """
    # Çıktı dosyasının adını oluştur
    output_file_path = os.path.splitext(input_file_path)[0] + '.wav'
    
    # Dosya zaten WAV formatında mı kontrol et
    if input_file_path.endswith('.wav'):
        print(f"'{input_file_path}' already in WAV format.")
        return input_file_path
    
    # FFmpeg komutunu oluştur
    command = ['ffmpeg', '-y', '-i', input_file_path, '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', output_file_path]

    # FFmpeg komutunu çalıştır
    try:
        ##print (command)
        print(command)
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Conversion successful: '{input_file_path}' to '{output_file_path}'")
        return output_file_path
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")
        return None

In [20]:
async def video_indir_ve_wav_donustur(url, hedef_klasor='indirilenler'):
    if not os.path.exists(hedef_klasor):
        os.makedirs(hedef_klasor)
    try:
        video = YouTube(url)
        ses_dosyasi = video.streams.filter(only_audio=True).first()
        indirme_yolu = ses_dosyasi.download(output_path=hedef_klasor)
        yeni_dosya_adi = convert_to_wav(indirme_yolu)
        os.remove(indirme_yolu)
        return yeni_dosya_adi
    except Exception as e:
        print(f"Video indirme ve dönüştürme sırasında hata oluştu: {e}")
        return None

In [21]:
async def sesi_isle_ve_bol(dosya_adi, sinif, index, sr=16000):
    #y, sr = librosa.load(dosya_adi, sr=sr)
    #y_reduced_noise = nr.reduce_noise(y=y, sr=sr)

    # İşlenmiş sesi geçici bir dosyaya kaydet
    #temp_dosya = f'temp_{sinif}_{index}.wav'
    #sf.write(temp_dosya, y_reduced_noise, sr)  # soundfile ile kaydet

    # 30 saniyelik segmentlere böl
    ses = AudioSegment.from_wav(dosya_adi)
    parca_uzunlugu = 30 * 1000
    for i in range(0, len(ses), parca_uzunlugu):
        parca = ses[i:i + parca_uzunlugu]
        if(parca.duration_seconds < parca_uzunlugu/1000):
            continue # 30 saniyeden kısa parçaları atla
        parca_adi = f"./../Data/{sinif}/{sinif}_{index}_{i//parca_uzunlugu}.wav"
        if not os.path.exists(f"./../Data/{sinif}"):
            os.makedirs(f"./../Data/{sinif}")
        parca.export(parca_adi, format="wav")

    #os.remove(temp_dosya)  # Geçici dosyayı sil

In [22]:
#CVS dosyasını oku ve her bir video için ses dosyasını indir, işle ve böl asenkron olarak 
import asyncio
import concurrent.futures

async def DownlandVideo(video, index):
    video_link, video_title, tag = video
    print(f"{index}. Video: {video_title}")
        
    ses_dosyasi = await video_indir_ve_wav_donustur(video_link, f'indirilenler/{tag}')
    if ses_dosyasi:
        await sesi_isle_ve_bol(ses_dosyasi, tag, index)
        print(f"{index}. Video indirildi ve WAV formatına dönüştürüldü.")
        return f"{index}. Video indirildi ve WAV formatına dönüştürüldü."
    else:
        print(f"{index}. Video indirme ve dönüştürme sırasında hata oluştu.")
        return f"{index}. Video indirme ve dönüştürme sırasında hata oluştu."




        

In [23]:

async def task1():
    with open('youtube_videos.csv', 'r', newline='', encoding='utf-8') as file:
        videos = list(csv.reader(file))
        #tag i AI COVER olanları atla
        videos = [video for video in videos if video[2] != 'AI COVER']
        for i, video in enumerate(videos):
            if i == 0:  # Başlıkları atla
                continue
            #create a task
            task = asyncio.create_task(DownlandVideo(video, i))
            task.add_done_callback(lambda t: print(t.result()))
            await task

async def task2():
    with open('youtube_videos.csv', 'r', newline='', encoding='utf-8') as file:
        videos = list(csv.reader(file))
        #tag i AI COVER olanları atla
        videos = [video for video in videos if video[2] != 'NON AI COVER']
        for i, video in enumerate(videos):
            if i == 0:  # Başlıkları atla
                continue
            #create a task
            task = asyncio.create_task(DownlandVideo(video, i))
            task.add_done_callback(lambda t: print(t.result()))
            await task




In [24]:
path = "./indirilenler"

#her bir klasör bir tagı temsil eder
tags = os.listdir(path)
for tag in tags:
    files = os.listdir(f"{path}/{tag}")
    index = 0
    for file in files:
        tpath = f"{path}/{tag}/{file}"
        await sesi_isle_ve_bol(tpath, tag, index)
        index += 1